# Script to compute the kinematics (for the first time)

In [8]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font
from scipy.interpolate import interp1d

from stressgait_analysis.dataset import StressGaitDataset

from src.stressgait_analysis.gait_helper_functions import compute_leg_kinematics, compute_arm_kinematics

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS, resample
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

In [10]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=True)

In [11]:

data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):

        cycle_start = hs_events[i]
        if cycle_start < 30:
            continue
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])
        arm_kinematics = compute_arm_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)
        arm_kinematics = resample(arm_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, j, *leg_row, *arm_row)
                for j, (leg_row, arm_row) in enumerate(zip(leg_kinematics.values, arm_kinematics.values))
                ])
        n_cycle += 1

In [12]:
kinematics_df = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns, *arm_kinematics.columns],).drop(columns = "ankle_plantarflexion")
kinematics_df.set_index([*dataset.index.columns, "cycle_idx", "percentage_of_stride"], inplace=True)

In [13]:
kinematics_df

hip_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                       15.617907   
                                           1                       14.898942   
                                           2                       14.222986   
                                           3                       13.644998   
                                           4                       13.199635   
...                                                                      ...   
VP_47       omc       2    fast  200       96                      19.179123   
                                           97                      19.724742   
                                           98                      20.170698   
                                           99                      20.258313   
                                           100                     20.283606   

                                                                 knee_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                 
VP_01       omc       1    slow  0         0                         5.708224   
                                           1                         5.079829   
                                           2                         4.740407   
                                           3                         5.059201   
                                           4                         5.675398   
...                                                                       ...   
VP_47       omc       2    fast  200       96                       13.579015   
                                           97                       11.883767   
                                           98                       10.334821   
                                           99                        9.310462   
                                           100                       8.377337   

                                                                 elbow_angle  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                      175.770471   
                                           1                      176.342514   
                                           2                      176.848315   
                                           3                      177.203232   
                                           4                      177.406398   
...                                                                      ...   
VP_47       omc       2    fast  200       96                     168.330404   
                                           97                     168.646740   
                                           98                     168.890194   
                                           99                     168.931013   
                                           100                    168.936592   

                                                                 shoulder_angle  \
participant condition bout speed cycle_idx percentage_of_stride                   
VP_01       omc       1    slow  0         0                         166.130160   
                                           1                         166.070783   
                                           2                         166.060716   
                                           3                         166.162965   
                                           4                         166.346609   
...                                                                         ...   
VP_47       omc       2    fast  200       96                        165.849253   
                                           97                        165.173631   
                                           98                        164.554919   
                

## store the data

In [14]:
filename = dataset.base_path.joinpath("kinematics/kinematics.csv")
kinematics_df.to_csv(filename)


# Great! Kinematics can now be loaded directly from the dataset!
